<a href="https://colab.research.google.com/github/Aditya6122/HashtagifyMe/blob/main/HashtagGenerator_HashtagifyMe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers --quiet
!pip install datasets --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.3 MB/s eta 0:00:00


In [2]:
import io
import urllib
import PIL.Image
import torch
import datasets
import numpy as np
import requests
from datasets import load_dataset
from transformers import BlipForConditionalGeneration, AutoProcessor
from datasets.utils.file_utils import get_datasets_user_agent
from torch.utils.data import Dataset
from tqdm.notebook import tqdm_notebook
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

In [ ]:
dataset_name = "conceptual_captions"
labeled_config = "labeled"
whole_dataset = load_dataset(dataset_name,labeled_config,split='train')
subset_size = 15000
dataset = whole_dataset.select(range(subset_size))
dataset = dataset.train_test_split(test_size=0.10, shuffle=True, seed=42)
dataset['train'] = dataset['train'].train_test_split(test_size=0.12, shuffle=True, seed=42)
data = {}
data['train'] = dataset['train']['train']
data['eval'] = dataset['train']['test']
data['test'] = dataset['test']
data['train'].set_format("torch")
data['eval'].set_format("torch")
data['test'].set_format("torch")

Generating train split:   0%|          | 0/2007090 [00:00<?, ? examples/s]

In [ ]:
USER_AGENT = get_datasets_user_agent()
def fetch_single_image(image_url, timeout=10, retries=0):
    for _ in range(retries + 1):
        try:
            request = urllib.request.Request(
                image_url,
                data=None,
                headers={"user-agent": USER_AGENT},
            )
            with urllib.request.urlopen(request, timeout=timeout) as req:
                image = PIL.Image.open(io.BytesIO(req.read()))
            break
        except Exception:
            image = None
    return image

In [ ]:
class ConceptualCaptionCustom(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __getitem__(self, idx):
        item = self.dataset[idx]
        img_url = item['image_url']
        label = item['labels']
        img = fetch_single_image(img_url)

        if img is None:
            return None

        target = [label[0]]
        for i in label[1:]:
            target.append('[SEP]')
            target.append(i)

        target = ' '.join([i for i in target])
        item = {"image": img, "text": target}
        try:
            encoding = self.processor(images=item["image"], text=item["text"],return_tensors="pt")
            encoding = {k:v.squeeze() for k,v in encoding.items()}
            return encoding
        except:
            return None

    def __len__(self):
        return len(self.dataset)

In [ ]:
processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

In [ ]:
train_dataset = ConceptualCaptionCustom(data['train'], processor)
val_dataset = ConceptualCaptionCustom(data['eval'], processor)
test_dataset = ConceptualCaptionCustom(data['test'],processor)

In [ ]:
def collate_fn(batch, dataset, batch_size):
    pixel_values = []
    input_ids = []

    for i in batch:
        if(i is not None):
            pixel_values.append(i['pixel_values'])
            input_ids.append(i['input_ids'])

    missing = 0
    if(len(batch) != batch_size):
        missing = batch_size - len(batch)

    while missing!=0:
        rand_idx = np.random.randint(0, len(dataset))
        rand_ele = dataset[rand_idx]
        if(rand_ele != None):
            pixel_values.append(rand_ele['pixel_values'])
            input_ids.append(rand_ele['input_ids'])
            missing -=1
        else:
            print("missing_idx :",rand_idx)
            print("values_remaining :",missing)

    validated_batch = {}
    validated_batch['input_ids'] = pad_sequence(input_ids).permute(1,0)
    validated_batch['pixel_values'] = torch.stack(pixel_values)

    return validated_batch

In [ ]:
batch_size = 8
train_dataloader =  DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8,pin_memory=True, collate_fn = lambda batch :collate_fn(batch, train_dataset, batch_size=batch_size))
val_dataloader =  DataLoader(val_dataset, batch_size=batch_size, num_workers=2,pin_memory=True, collate_fn = lambda batch :collate_fn(batch, train_dataset, batch_size=batch_size))
test_dataloader =  DataLoader(test_dataset, batch_size=batch_size, num_workers=2,pin_memory=True, collate_fn = lambda batch :collate_fn(batch, train_dataset, batch_size=batch_size))

In [ ]:
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

In [ ]:
for param in model.parameters():
    param.requires_grad = True

for vision_param in model.vision_model.parameters():
    vision_param.requires_grad = False

bert_params = model.text_decoder.bert.parameters()
cls_params = model.text_decoder.cls.parameters()

optimizer = torch.optim.AdamW([
                {'params': bert_params, 'lr': 1e-5},
                {'params': cls_params}
            ], lr=1e-3)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

model.train()
num_epochs = 13
running_loss = 0
progress_bar = 0
subsection = 200

for epoch in range(num_epochs):
  print(f"Epoch [{epoch + 1}/{num_epochs}]")
  print("-"*100)
  for idx, batch in enumerate(train_dataloader):
    input_ids = batch["input_ids"].to(device)
    pixel_values = batch["pixel_values"].to(device)

    outputs = model(input_ids=input_ids,
                    pixel_values=pixel_values,
                    labels=input_ids)

    loss = outputs.loss
    running_loss += loss

    if idx % subsection == 0:
            if(progress_bar):
                progress_bar.close()
            print(f"Epoch [{epoch + 1}/{num_epochs}], Batch [{idx + 1}/{len(train_dataloader)}], Loss: {running_loss/subsection}")
            running_loss = 0
            progress_bar = tqdm_notebook(total=subsection)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    progress_bar.update(1)


In [ ]:
url = 'http://www.hawaiigolf.com/photo-galleries/images/preview/28848.jpg'
raw_image = PIL.Image.open(requests.get(url, stream=True).raw)
inputs = processor(raw_image, return_tensors="pt").to("cuda")
model.eval()
with torch.no_grad():
    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)
    print(caption)